In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Kafka-Spark Integration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lab12/.ivy2/cache
The jars for the packages stored in: /home/lab12/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f1789476-19ef-43ec-b0a7-b803c62ed72c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.2/spark-sql-kafka-0-10_2.12-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.

In [4]:

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

query = kafka_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

print("Streaming is running...")
query.awaitTermination()

24/12/18 13:37:10 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2270f2bf-30e6-4f54-9b74-ec3ecb0af5b0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Streaming is running...
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
| test|
+-----+

+-----+
|value|
+-----+
| test|
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
| test|
+-----+



KeyboardInterrupt: 

옵션 설정 테스트

In [3]:

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .option("startingOffsets", "earliest") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

query = kafka_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='3 seconds') \
    .start()

print("Streaming is running...")
query.awaitTermination()

24/12/18 13:36:47 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-022c8675-004b-4bce-bb27-318a35037692. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Streaming is running...


-------------------------------------------
Batch: 0
-------------------------------------------
+---------------+
|          value|
+---------------+
|hHEHello|
|         hello2|
|      Message 0|
|      Message 1|
|      Message 2|
|      Message 3|
|      Message 4|
|          world|
+---------------+



KeyboardInterrupt: 

In [ ]:
데이터 처리 테스트

In [ ]:

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "kafka-test") \
    .load()

kafka_data = kafka_stream.selectExpr("CAST(value AS STRING)")

# 모든 실행 중인 쿼리를 확인하고 중지
for query in spark.streams.active:
    if query.name == "kafka_messages":
        query.stop()
        
query = kafka_data.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("kafka_messages") \
    .start()

print("Streaming is running...")

import time
while True:
    spark.sql("SELECT * FROM kafka_messages").show()
    time.sleep(5)

In [5]:
spark.stop()